## EA 3 Object 1
## Upload json files, transform, merge and export to csv files



### Importing necessary libraries

In [1]:
import pandas as pd
import json
import csv
import warnings; warnings.simplefilter('ignore')

### Importing Json files in Pandas Dataframe

In [2]:
df_countries = pd.read_json('..\data\countries.json', encoding='LATIN1')
df_languages = pd.read_json('..\data\languages.json', encoding='LATIN1')

### Transponding the previous DataFrames

In [3]:
df_countries_2 = df_countries.T
df_languages_2 = df_languages.T

### Creating some headers dataframes that will help us parsing and structuring complete dataframe from json files

In [4]:
df_header_c = pd.DataFrame(columns=["country"])
for line in list(df_countries[:0]):
    x = str(line)
    df_header_c = df_header_c.append({"country": x}, ignore_index=True)
       

In [5]:
df_header_l = pd.DataFrame(columns=["country"])
for line in list(df_languages[:0]):
    x = str(line)
    df_header_l = df_header_l.append({"country": x}, ignore_index=True)


In [6]:
df_header_c.head()

,country
0,AD
1,AE
2,AF
3,AG
4,AI


In [7]:
df_header_l.head()

,country
0,aa
1,ab
2,af
3,ak
4,am


### Reseting indexes for all dataframes will allow us for adding ordered header as a new column

In [8]:
df_countries_2.reset_index(drop=True, inplace=True)
df_header_c.reset_index(drop=True, inplace=True)
df_languages_2.reset_index(drop=True, inplace=True)
df_header_l.reset_index(drop=True, inplace=True)

### We add the new column "country" to the original datafrma

In [9]:
df_countries_3 = pd.concat([df_countries_2, df_header_c], axis=1)
df_languages_3 = pd.concat([df_languages_2, df_header_c], axis=1)

In [10]:
df_countries_3.head()

,name,native,phone,continent,capital,currency,languages,country
0,Andorra,Andorra,376,EU,Andorra la Vella,EUR,[ca],AD
1,United Arab Emirates,Ø¯ÙÙØ© Ø§ÙØ¥Ù Ø§Ø±Ø§Øª Ø§ÙØ¹Ø±Ø¨ÙØ© Ø§ÙÙ...,971,AS,Abu Dhabi,AED,[ar],AE
2,Afghanistan,Ø§ÙØºØ§ÙØ³ØªØ§Ù,93,AS,Kabul,AFN,"[ps, uz, tk]",AF
3,Antigua and Barbuda,Antigua and Barbuda,1268,NA,Saint John's,XCD,[en],AG
4,Anguilla,Anguilla,1264,NA,The Valley,XCD,[en],AI


### Merging the previous dataframes : languages and countries by country

In [11]:
df_merged = pd.merge(df_countries_3, df_languages_3, how='inner', on='country')

### When merging, we generate duplicated namen columns _x _y
### This piece of code filter those duplicates and rename columns to original naming

In [12]:
# define our drop function
def drop_y(df):
    # list comprehension of the cols that end with '_y'
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(to_drop, axis=1, inplace=True)

In [13]:
def rename_x(df):
    for col in df:
        if col.endswith('_x'):
            df.rename(columns={col:col.rstrip('_x')}, inplace=True)

In [14]:
drop_y(df_merged)
rename_x(df_merged)

### Creating a new dataframe with the columns needed

In [15]:
df_merged_2 = df_merged[['name', 'continent', 'country','languages']]


### As we can see, in the column languages we have lists of languages

In [16]:
df_merged_2

,name,continent,country,languages
0,Andorra,EU,AD,[ca]
1,United Arab Emirates,AS,AE,[ar]
2,Afghanistan,AS,AF,"[ps, uz, tk]"
3,Antigua and Barbuda,NA,AG,[en]
4,Anguilla,NA,AI,[en]
...,...,...,...,...
245,Yemen,AS,YE,[ar]
246,Mayotte,AF,YT,[fr]
247,South Africa,AF,ZA,"[af, en, nr, st, ss, tn, ts, ve, xh, zu]"
248,Zambia,AF,ZM,[en]


### We need to parse the column languages and generate new columns "flag_"+ language for the different values

In [17]:
list_langs = list(df_merged_2['languages'])

In [18]:
unique_langs = set(x for l in list_langs for x in l)


In [19]:
for line in unique_langs:
        string = "flag_"+line
        df_merged_2[string] = df_merged_2["languages"].apply(lambda x:1 if line in x else 0)

### We add a new column adding the previous generated flag_ + language

In [20]:
sum_langs = df_merged_2.filter(regex="flag_") 

df_merged_2['total_langs'] = sum_langs.sum(axis=1) 


### Finally we export the merged dataframe with the needed columns to csv file

In [21]:
df_merged_2.to_csv ('../output_data/DF_Merged.csv', index = None, header=True) 